# Season 15 Election Notes

>**FORBIDDEN KNOWLEDGE WARNING**
>
>This document contains some information that is not currently present on the main site and may be considered spoilers.
>Continue at your own discretion. Some information in this document cannot be shared on the Discord without spoiler tags.

In [458]:
%matplotlib inline
from blaseball_mike.models import *
import plotly.express as plot
import plotly.io as _pio
import plotly.subplots as subplot
from IPython.display import display, Markdown

import os
import sys
pdir = os.path.abspath(os.path.join(os.path.dirname(''), os.path.pardir))
sys.path.append(pdir)
from display import *
from blessings import *
sys.path.remove(pdir)

_pio.renderers.default = "notebook_connected"

pies = Team.load_by_name("Philly Pies")

In [459]:
sim = SimulationData.load()
print(f"Last Updated Season {sim.season}, Day {sim.day}")

Last Updated Season 15, Day 28


---
## Decrees

Decrees are chosen by majority votes across all teams. `1` decree will pass this season.

### Bats/Caps/Shoes/Gloves
**Swing Away. Go on the offensive while providing a Global Hitting Boost.**

**Don't Lose Your Heads. Keep your wits about you while providing a global Pitching Boost.**

**Flight or Flight. Run away while providing global Baserunning Boost.**

**Catch and Release. Keep threats in front of you while providing a global Defense Boost.**

All this season's decrees result in a league-wide boost to various stats. Below is a table of all the teams average
stars in each stat for comparison.


In [460]:
league = League.load()
all_teams = league.teams

league_stats = pandas.DataFrame()
for team in all_teams.values():
    team_stats = pandas.Series({"batting": mean([x.batting_rating*5 for x in team.lineup]),
                                "pitching": mean([x.pitching_rating*5 for x in team.rotation]),
                                "baserunning": mean([x.baserunning_rating*5 for x in team.lineup]),
                                "defense": mean([x.defense_rating*5 for x in team.lineup])},
                               name=team.nickname)
    league_stats = league_stats.append(team_stats)
set_heatmap(league_stats, maxVal=5)

,baserunning,batting,defense,pitching
Lift,3.683928,3.531344,3.355274,2.957322
Jazz Hands,3.281196,3.227272,3.260406,3.334229
Firefighters,2.676537,3.400876,3.280034,3.557141
Tigers,3.176052,3.572320,2.656421,3.904019
Wild Wings,2.480337,2.910656,3.081335,4.218983
Georgias,3.342497,3.390911,3.381689,3.236687
Tacos,3.225578,3.326464,3.459332,3.734981
Dale,3.182297,3.546013,3.117036,4.139326
Flowers,3.310812,3.752268,2.774682,3.565013
Sunbeams,2.997740,3.605952,3.219914,3.014362


___
## Wills

Will are selected for each team by raffle. Each team will draw `2` random wills. The bottom 4 teams in the league by
standings will draw `3`. The chosen wills are random, but wills with more votes are more likely to be selected.

### Plunder
**Steal a Player from the Idols board. Send a random Player from your Team back.**

Due to the random nature of the target on our team, this tests the best case (swapping the Best Player on the Idol Board
for a position with our worst player in that position) and the worst case (swapping the Best Player on the Idol Board
for a position with our best player in that position).

**Best Case Swap:**

In [461]:
idols = [x.player for x in Idol.load().values() if x.player.league_team_id != pies]

idols.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_idol = idols[0]
idols.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_idol = idols[0]

worst_batter = sort_lineup(pies, 1)[0]
print(f"Batting: Swap {best_batting_idol.name} for {worst_batter.name}")

pandas.DataFrame(trade_players(pies, worst_batter, best_batting_idol))

Batting: Swap Nagomi Mcdaniel for -u-e- P-ddles


,Pies
batting_change,+11.7078%
pitching_change,+0.0000%
baserunning_change,+7.5644%
defense_change,+3.3510%


In [462]:
worst_pitcher = sort_rotation(pies, 1)[0]
print(f"Pitching: Swap {best_pitching_idol.name} for {worst_pitcher.name}")

pandas.DataFrame(trade_players(pies, worst_pitcher, best_pitching_idol))


Pitching: Swap Nagomi Mcdaniel for Doc Anice


,Pies
batting_change,+0.0000%
pitching_change,+15.3801%
baserunning_change,+0.0000%
defense_change,+0.0000%


**Worst Case Swap:**

In [463]:
best_batter = sort_lineup(pies, 1, order="best")[0]
print(f"Batting: Swap {best_batting_idol.name} for {best_batter.name}")

pandas.DataFrame(trade_players(pies, best_batter, best_batting_idol))

Batting: Swap Nagomi Mcdaniel for J-xo- B-c--ey


,Pies
batting_change,-0.2079%
pitching_change,+0.0000%
baserunning_change,+3.6755%
defense_change,+5.0798%


In [464]:
best_pitcher = sort_rotation(pies, 1, order="best")[0]
print(f"Pitching: Swap {best_pitching_idol.name} for {best_pitcher.name}")

pandas.DataFrame(trade_players(pies, best_pitcher, best_pitching_idol))


Pitching: Swap Nagomi Mcdaniel for Elvis Figueroa


,Pies
batting_change,+0.0000%
pitching_change,+4.8122%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Equivalent Exchange
**Swap a Player from a Team in the opposite Subleague with a Player from your Team. They must be within 2 combined stars of each other.**

See below for the 10 best swap candidates for a particular player, sorted by highest stats in the applicable position.

In [465]:
wild = Subleague.load("aabc11a1-81af-4036-9f18-229c759ca8a9")
all_wild = []
for team in wild.teams.values():
    all_wild.extend(team.lineup + team.rotation)

In [466]:
all_total_stars = get_total_stars(all_wild)
for player in pies.lineup:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_wild if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.batting_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:10]))
    display(set_heatmap(get_batting_stlats([player] + player_matches[0:10]), maxVal=1))

####Eduardo Woodman

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,4.0,2.0,3.5,3.5
Jacob Haynes,5.5,3.0,2.0,1.5
Ve--ali-s Sund-e,5.5,3.0,3.5,3.5
Baby Doyle,5.0,1.5,2.5,4.0
Math Velazquez,5.0,3.5,4.0,2.0
Comfort Septemberish,5.0,1.0,5.0,2.5
Scores Baserunner,4.5,1.5,3.0,4.0
Bauer Zimmerman,4.5,1.5,3.0,2.5
Cannonball Sports,4.5,1.0,3.0,3.5
Brock Watson,4.5,3.0,4.0,4.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.833060,1.019391,0.846540,0.650666,0.575519,0.595517,0.574554,0.100000,0.705869
Jacob Haynes,1.066191,1.263815,0.942263,1.101913,1.219008,0.001000,0.396483,0.060000,0.823724
Ve--ali-s Sund-e,1.061020,0.887561,1.410623,0.561752,1.295647,0.001000,1.249409,0.001000,0.612723
Baby Doyle,1.018570,1.223064,0.962650,1.268464,0.584158,0.311593,1.156570,0.049795,1.208599
Math Velazquez,0.981884,1.027667,1.067639,0.709586,1.146343,0.395484,0.622834,0.060000,0.682863
Comfort Septemberish,0.978574,1.177996,1.187902,0.375357,0.767285,0.611622,1.105479,0.060000,0.832023
Scores Baserunner,0.919824,1.152613,0.934063,0.610101,0.478674,0.001000,0.426670,0.001000,0.369985
Bauer Zimmerman,0.884063,0.963640,0.967942,0.413869,0.969917,0.426705,0.926229,0.100000,0.225597
Cannonball Sports,0.875918,0.921890,0.873335,0.797885,0.804610,0.387646,1.066485,0.013993,1.006218
Brock Watson,0.874652,0.503273,1.309383,1.178311,1.250319,0.216144,0.785394,0.001000,1.081014


####Beasley Day

,Batting,Pitching,Baserunning,Defense
Beasley Day,4.0,2.5,3.5,2.5
Jacob Haynes,5.5,3.0,2.0,1.5
Baby Doyle,5.0,1.5,2.5,4.0
Math Velazquez,5.0,3.5,4.0,2.0
Comfort Septemberish,5.0,1.0,5.0,2.5
Scores Baserunner,4.5,1.5,3.0,4.0
Bauer Zimmerman,4.5,1.5,3.0,2.5
Cannonball Sports,4.5,1.0,3.0,3.5
Baby Triumphant,4.5,1.5,2.0,3.0
Reese Clark,4.5,3.0,2.0,4.0


...

,Batting,Pitching,Baserunning,Defense
-u-e- P-ddles,2.0,2.5,3.0,3.5
Jacob Haynes,5.5,3.0,2.0,1.5
Bauer Zimmerman,4.5,1.5,3.0,2.5
Cannonball Sports,4.5,1.0,3.0,3.5
Baby Triumphant,4.5,1.5,2.0,3.0
Sum--rs -resto-,4.5,2.5,2.5,2.5
Peanutiel Duffy,4.0,3.5,1.5,3.5
August Sky,4.0,3.5,1.5,2.0
-a- M-son,4.0,1.5,2.5,3.0
Paula Turnip,4.0,1.0,3.5,3.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
-u-e- P-ddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015
Jacob Haynes,1.066191,1.263815,0.942263,1.101913,1.219008,0.001000,0.396483,0.060000,0.823724
Bauer Zimmerman,0.884063,0.963640,0.967942,0.413869,0.969917,0.426705,0.926229,0.100000,0.225597
Cannonball Sports,0.875918,0.921890,0.873335,0.797885,0.804610,0.387646,1.066485,0.013993,1.006218
Baby Triumphant,0.871939,0.888347,1.040411,0.562577,0.678567,0.010000,0.167941,0.100000,0.788547
Sum--rs -resto-,0.858482,0.836937,1.070270,1.088566,0.230245,0.052847,0.685552,0.001000,0.731691
Peanutiel Duffy,0.841420,1.038584,0.738454,0.996131,0.670202,0.418180,0.343720,0.100000,0.240592
August Sky,0.841003,0.956374,0.827577,0.870913,0.642353,0.393628,0.326945,0.036738,0.331171
-a- M-son,0.831824,1.089116,0.635958,0.783859,0.732174,0.207444,0.899090,0.019604,0.485050
Paula Turnip,0.816815,0.766132,1.017299,0.583585,0.885149,0.544858,0.284335,0.100000,0.261251


####Jessica Telephone

,Batting,Pitching,Baserunning,Defense
Jessica Telephone,5.5,2.5,4.0,3.0
Nagomi Nava,5.5,2.5,4.5,3.0
Ve--ali-s Sund-e,5.5,3.0,3.5,3.5
Dudley Mueller,5.0,3.0,3.5,4.5
Margarito Nava,5.0,3.5,4.0,2.5
Gerund Pantheocide,5.0,3.5,3.0,4.0
Baby Doyle,5.0,1.5,2.5,4.0
Math Velazquez,5.0,3.5,4.0,2.0
Comfort Septemberish,5.0,1.0,5.0,2.5
Logan Horseman,5.0,3.0,5.0,3.5


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Jessica Telephone,1.129590,1.113282,1.216455,1.326294,0.982475,0.215902,1.574396,0.100000,0.919039
Nagomi Nava,1.074517,0.736258,1.654306,0.761977,1.582309,0.111803,0.769464,0.001000,0.917821
Ve--ali-s Sund-e,1.061020,0.887561,1.410623,0.561752,1.295647,0.001000,1.249409,0.001000,0.612723
Dudley Mueller,1.048644,1.565780,0.771321,0.945664,1.004410,0.223640,0.931322,0.060000,1.553180
Margarito Nava,1.039012,0.828340,1.284494,1.284454,1.444628,0.482716,1.171332,0.001000,0.826623
Gerund Pantheocide,1.027445,1.300367,1.034786,0.893386,0.575666,0.270360,0.571867,0.001000,0.289087
Baby Doyle,1.018570,1.223064,0.962650,1.268464,0.584158,0.311593,1.156570,0.049795,1.208599
Math Velazquez,0.981884,1.027667,1.067639,0.709586,1.146343,0.395484,0.622834,0.060000,0.682863
Comfort Septemberish,0.978574,1.177996,1.187902,0.375357,0.767285,0.611622,1.105479,0.060000,0.832023
Logan Horseman,0.957968,1.295582,0.833595,0.491691,1.152161,0.417793,0.990140,0.001000,0.672432


In [467]:
for player in pies.rotation:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_wild if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.pitching_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:10]))
    display(set_heatmap(get_pitching_stlats([player] + player_matches[0:10]), maxVal=1))

####Henry Marshallow

,Batting,Pitching,Baserunning,Defense
Henry Marshallow,3.5,3.5,2.5,4.5
Rafael Davids,3.5,5.0,2.0,3.5
-----ide- B-n--e,3.5,4.5,4.5,2.5
Xandra Pancakes,2.0,4.5,3.5,4.0
Cory Twelve,3.5,4.5,3.5,2.5
Wyatt Pothos,2.5,4.0,3.5,3.0
Cell Barajas,3.0,4.0,2.5,3.0
Engine Eberhardt,3.0,4.0,5.0,3.5
Mummy Melcon,4.0,4.0,3.5,3.0
Jayden Wright,2.5,4.0,2.5,3.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.687948,0.423292,1.106023,1.381125,0.774138,0.745639,0.821668
Rafael Davids,0.950434,0.821531,1.097610,1.057661,1.006934,1.045070,1.161455
-----ide- B-n--e,0.944823,0.944301,1.093810,0.979122,0.575652,0.799962,0.643485
Xandra Pancakes,0.880726,0.996416,0.916019,0.776169,0.754645,0.383383,0.626532
Cory Twelve,0.869612,0.997242,1.011806,0.635079,0.472467,1.002317,0.191589
Wyatt Pothos,0.844384,0.701425,1.063023,1.132335,0.691922,1.079506,0.799864
Cell Barajas,0.819420,0.927790,0.968070,0.718913,0.614807,0.132356,0.139887
Engine Eberhardt,0.804873,1.086360,0.715832,0.543387,0.776343,0.727395,0.709508
Mummy Melcon,0.794301,0.556494,1.131270,0.909441,1.306716,1.036951,0.937577
Jayden Wright,0.786132,0.774936,0.845408,0.756174,1.008662,0.821718,0.359170


####Nerd Pacheco

,Batting,Pitching,Baserunning,Defense
Nerd Pacheco,4.5,4.0,4.5,3.0
Burke Gonzales,3.5,6.0,4.0,4.0
Rigby Friedrich,3.5,5.0,4.5,4.0
Lou Roseheart,3.5,5.0,4.5,4.0
Rafael Davids,3.5,5.0,2.0,3.5
-----ide- B-n--e,3.5,4.5,4.5,2.5
Yummy Elliott,3.5,4.5,4.0,4.0
Xandra Pancakes,2.0,4.5,3.5,4.0
Yurts Buttercup,4.0,4.5,3.5,4.0
Engine Eberhardt,3.0,4.0,5.0,3.5


...

,Batting,Pitching,Baserunning,Defense
Ruslan Greatness,5.0,4.0,5.0,5.0
Castillo Turner,5.5,6.0,4.5,4.5
Pitching Machine,4.5,6.0,3.5,4.0
Burke Gonzales,3.5,6.0,4.0,4.0
Qais Dogwalker,4.5,6.0,5.5,5.0
Gabriel Griffith,5.0,5.5,5.0,5.0
Rigby Friedrich,3.5,5.0,4.5,4.0
Wesley Poole,3.5,3.5,5.5,5.0
Rivers Clembons,4.5,3.5,4.5,4.5
Loubert Ji-Eun,4.5,3.5,5.5,4.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Ruslan Greatness,0.819339,0.770870,0.786481,1.269963,1.026832,0.629742,1.420762
Castillo Turner,1.212347,1.201898,1.252195,0.810007,1.390535,1.449835,1.080382
Pitching Machine,1.211224,1.370252,1.240744,1.196028,0.506950,0.642323,0.891417
Burke Gonzales,1.176989,1.171639,1.341412,0.879956,0.870391,0.973638,0.789646
Qais Dogwalker,1.153869,1.071425,1.137700,1.358705,1.152111,0.882765,0.610121
Gabriel Griffith,1.106354,1.217163,1.327721,0.656127,0.669855,0.746840,0.663962
Rigby Friedrich,0.961982,1.119728,1.004998,0.497562,1.143943,0.785381,0.584288
Wesley Poole,0.748505,0.956576,0.792677,0.347421,0.964891,0.609278,0.765654
Rivers Clembons,0.715612,0.765694,0.630133,0.923867,1.147302,0.481627,1.144964
Loubert Ji-Eun,0.714377,0.809874,0.571582,1.254389,0.738529,0.647790,1.006571


####Elvis Figueroa

,Batting,Pitching,Baserunning,Defense
Elvis Figueroa,3.5,5.5,2.0,4.5
Burke Gonzales,3.5,6.0,4.0,4.0
Rigby Friedrich,3.5,5.0,4.5,4.0
Lou Roseheart,3.5,5.0,4.5,4.0
Rafael Davids,3.5,5.0,2.0,3.5
-----ide- B-n--e,3.5,4.5,4.5,2.5
Yummy Elliott,3.5,4.5,4.0,4.0
Xandra Pancakes,2.0,4.5,3.5,4.0
Yurts Buttercup,4.0,4.5,3.5,4.0
Engine Eberhardt,3.0,4.0,5.0,3.5


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Elvis Figueroa,1.096633,1.159675,1.136143,1.072682,0.786089,0.461405,0.721203
Burke Gonzales,1.176989,1.171639,1.341412,0.879956,0.870391,0.973638,0.789646
Rigby Friedrich,0.961982,1.119728,1.004998,0.497562,1.143943,0.785381,0.584288
Lou Roseheart,0.961006,1.019976,0.835425,1.001713,1.215511,1.104658,1.076313
Rafael Davids,0.950434,0.821531,1.097610,1.057661,1.006934,1.045070,1.161455
-----ide- B-n--e,0.944823,0.944301,1.093810,0.979122,0.575652,0.799962,0.643485
Yummy Elliott,0.912388,0.798342,1.147276,1.108293,0.617898,0.948697,0.932231
Xandra Pancakes,0.880726,0.996416,0.916019,0.776169,0.754645,0.383383,0.626532
Yurts Buttercup,0.852575,0.987203,1.095826,0.627572,0.287007,1.223968,0.584248
Engine Eberhardt,0.804873,1.086360,0.715832,0.543387,0.776343,0.727395,0.709508


### Foreshadow
**Swap a Player from your Shadows to your active roster.**

There are 14 shadows players to swap with our 12 existing players. Below is a breakdown comparing the best shadows
players with the worst player in the particular position.

Note that position does not matter for the swap: any shadows player can bat or pitch.


**Batting:**

In [468]:
shadows = pies.bullpen + pies.bench
shadows.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_shadows = shadows[0:4]
worst_batter = sort_lineup(pies, 1)[0]

get_stars(best_batting_shadows + [worst_batter])

,Batting,Pitching,Baserunning,Defense
Haruta Byrd,3.5,2.5,2.5,1.5
Javier Lotus,3.5,1.0,3.5,4.0
August Obrien,3.5,1.0,3.5,2.0
Jefferson Delacruz,3.0,0.5,3.5,4.0
-u-e- P-ddles,2.0,2.5,3.0,3.5


In [469]:
set_heatmap(get_batting_stlats(best_batting_shadows + [worst_batter]), maxVal=1)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Haruta Byrd,0.724391,0.721433,0.917077,0.853010,0.585921,0.832203,0.169263,0.100000,0.124161
Javier Lotus,0.682621,0.938964,0.386700,0.901328,0.942211,0.205027,0.884526,0.100000,0.815592
August Obrien,0.681669,0.641217,0.927569,0.913891,0.492708,0.923854,0.556849,0.100000,0.463958
Jefferson Delacruz,0.595750,0.479940,0.643757,0.903986,0.537770,0.503675,0.434424,0.100000,0.671689
-u-e- P-ddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015


**Pitching:**

In [470]:
shadows.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_shadows = shadows[0:4]
worst_pitcher = sort_rotation(pies, 1)[0]

get_stars(best_pitching_shadows + [worst_pitcher])

,Batting,Pitching,Baserunning,Defense
Lucy Tokkan,2.0,3.5,1.0,3.0
Twofurious Puddles,2.5,3.5,2.0,3.5
Case Lancaster,0.0,3.5,2.0,3.0
Tiana Takahashi,1.0,3.0,2.5,3.0
Doc Anice,2.5,3.0,1.5,3.5


In [471]:
set_heatmap(get_pitching_stlats(best_pitching_shadows + [worst_pitcher]), maxVal=1)

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Lucy Tokkan,0.741748,0.901966,0.995562,0.263758,0.973020,0.180980,0.321805
Twofurious Puddles,0.694922,0.811110,0.650034,0.807712,0.718615,0.416240,0.725894
Case Lancaster,0.658365,0.867044,0.829197,0.190119,0.825333,0.868251,0.082117
Tiana Takahashi,0.618850,0.910065,0.961592,0.716677,0.028954,0.596122,0.945940
Doc Anice,0.568234,0.928114,0.359045,0.836037,0.456020,0.598965,0.144698


### Move
**Move a Player on your Team to the desired location on your Roster.**

Move would swap a player from any location on the team (including Shadows) to another position, increasing the number
of players in that position. See below for moving our best hitting pitcher or best pitching hitter into their preferred
position.

**Batting**

In [472]:
bhp = best_hitting_pitcher(pies)
bph = best_pitching_hitter(pies)

print(f"Swapping {bhp.name} with {worst_batter.name}")
pandas.DataFrame(swap_players(pies, bhp, worst_batter))

Swapping Ruslan Greatness with -u-e- P-ddles


,Pies
batting_change,+7.4505%
pitching_change,-6.9715%
baserunning_change,+6.7042%
defense_change,+4.1826%


In [473]:
print(f"Swapping {bph.name} with {worst_pitcher.name}")
pandas.DataFrame(swap_players(pies, bph, worst_pitcher))

Swapping J-xo- B-c--ey with Doc Anice


,Pies
batting_change,-11.1292%
pitching_change,+2.1257%
baserunning_change,-7.0486%
defense_change,+1.9766%


**Pitching**

### Transfuse
**Boost a Player on your Team by 15% to 30% in a Stat Category of your choice, but Impair them in a Random other Category.**

The stat that is negatively impacted is random. It is generally better to boost pitchers, as only the pitching stats are
used for that position. See below for how this would affect various team members at the 2 extreme percentages. This will
only affect one player.

**15% Change:**

In [474]:
infuse_lineup_table, _, _ = improve_team_batting_table(pies, 0.15)
infuse_lineup_table

,old_batting_stars,new_batting_stars,change_in_batting_stars
Eduardo Woodman,4.165300,4.885272,0.719972
Beasley Day,3.969778,4.706006,0.736228
Huber Frumple,3.806051,4.575154,0.769103
J-xo- B-c--ey,6.318317,6.956462,0.638145
Lang Richardson,3.162214,3.907698,0.745485
-u-e- P-ddles,2.147811,3.068746,0.920935
Jessica Telephone,5.647950,6.337272,0.689322


In [475]:
set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.15)), maxVal=1.0)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.977054,1.169391,0.996540,0.800666,0.725519,0.445517,0.724554,0.010000,0.855869
Beasley Day,0.941201,1.124873,1.000851,0.450545,0.651918,0.122384,0.836625,0.010000,1.115368
Huber Frumple,0.915031,0.573284,1.319818,1.027497,1.347390,0.195580,0.794357,0.010000,1.213890
J-xo- B-c--ey,1.391292,1.643269,1.137449,1.826092,2.002606,0.010000,2.082640,0.010000,1.477099
Lang Richardson,0.781540,0.962722,0.550621,0.772843,1.091821,0.055246,0.648400,0.010000,0.701015
-u-e- P-ddles,0.613749,1.183698,0.334577,0.825196,0.301908,0.480778,0.260821,0.010000,1.160015
Jessica Telephone,1.267454,1.263282,1.366455,1.476294,1.132475,0.065902,1.724396,0.010000,1.069039


In [476]:
infuse_rotation_table, _, _ = improve_team_pitching_table(pies, 0.15)
infuse_rotation_table


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,3.439741,4.374069,0.934328
Nerd Pacheco,4.180600,5.151709,0.971109
Doc Anice,2.841172,3.734213,0.893041
Ruslan Greatness,4.096695,4.976003,0.879308
Elvis Figueroa,5.483164,6.399624,0.916460


In [477]:
set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.15)), maxVal=1)


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.874814,0.573292,1.256023,1.531125,0.924138,0.895639,0.971668
Nerd Pacheco,1.030342,1.237720,1.216351,0.422113,0.758134,1.082212,0.634500
Doc Anice,0.746843,1.078114,0.509045,0.986037,0.606020,0.748965,0.294698
Ruslan Greatness,0.995201,0.920870,0.936481,1.419963,1.176832,0.779742,1.570762
Elvis Figueroa,1.279925,1.309675,1.286143,1.222682,0.936089,0.611405,0.871203


**30% Change:**

In [478]:
infuse_lineup_table, _, _ = improve_team_batting_table(pies, 0.30)
infuse_lineup_table

,old_batting_stars,new_batting_stars,change_in_batting_stars
Eduardo Woodman,4.165300,5.585918,1.420618
Beasley Day,3.969778,5.408314,1.438535
Huber Frumple,3.806051,5.311149,1.505098
J-xo- B-c--ey,6.318317,7.586190,1.267873
Lang Richardson,3.162214,4.606233,1.444019
-u-e- P-ddles,2.147811,3.885398,1.737587
Jessica Telephone,5.647950,6.981993,1.334043


In [479]:
set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.30)), maxVal=1)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.117184,1.319391,1.146540,0.950666,0.875519,0.295517,0.874554,0.010000,1.005869
Beasley Day,1.081663,1.274873,1.150851,0.600545,0.801918,0.010000,0.986625,0.010000,1.265368
Huber Frumple,1.062230,0.723284,1.469818,1.177497,1.497390,0.045580,0.944357,0.010000,1.363890
J-xo- B-c--ey,1.517238,1.793269,1.287449,1.976092,2.152606,0.010000,2.232640,0.010000,1.627099
Lang Richardson,0.921247,1.112722,0.700621,0.922843,1.241821,0.010000,0.798400,0.010000,0.851015
-u-e- P-ddles,0.777080,1.333698,0.484577,0.975196,0.451908,0.330778,0.410821,0.010000,1.310015
Jessica Telephone,1.396399,1.413282,1.516455,1.626294,1.282475,0.010000,1.874396,0.010000,1.219039


In [480]:
infuse_rotation_table, _, _ = improve_team_pitching_table(pies, 0.30)
infuse_rotation_table


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,3.439741,5.311955,1.872214
Nerd Pacheco,4.180600,6.110125,1.929525
Doc Anice,2.841172,4.636189,1.795017
Ruslan Greatness,4.096695,5.876536,1.779841
Elvis Figueroa,5.483164,7.332289,1.849125


In [481]:
set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.30)), maxVal=1)


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,1.062391,0.723292,1.406023,1.681125,1.074138,1.045639,1.121668
Nerd Pacheco,1.222025,1.387720,1.366351,0.572113,0.908134,1.232212,0.784500
Doc Anice,0.927238,1.228114,0.659045,1.136037,0.756020,0.898965,0.444698
Ruslan Greatness,1.175307,1.070870,1.086481,1.569963,1.326832,0.929742,1.720762
Elvis Figueroa,1.466458,1.459675,1.436143,1.372682,1.086089,0.761405,1.021203


### Alternate Trust
**Call in an Alternate for a Player on your Team, with a mystery Benefit.**

An Alternate is a player reroll and the Benefit is an unknown modification. Below shows the change in Team Average Stars from rerolling our worst
player, assuming they would be replaced with the Statistically Average player.

In [482]:
worst_pie = sort_overall(pies, 1)[0]

print(f"Rerolling {worst_pie.name}")
if worst_pie in pies.rotation:
    out = replace_player(pies, worst_pie, pitch_star=NEW_PLAYER["pitching"])
else:
    out = replace_player(pies, worst_pie, bat_star=NEW_PLAYER["batting"], run_star=NEW_PLAYER["baserunning"], def_star=NEW_PLAYER["defense"])
pandas.DataFrame(out)

Rerolling -u-e- P-ddles


,Pies
batting_change,-0.4223%
pitching_change,+0.0000%
baserunning_change,-0.9070%
defense_change,-3.1326%


### Reform
**Re-roll an eligible Permanent Modification for a Player on your Team.**

Below is a list of all players with modifications that could be rerolled. It would be better to reroll modifications
with negative effects.

In [483]:
mod_players = [x for x in pies.lineup + pies.rotation if len(x.perm_attr) > 0]
mod_list = pandas.DataFrame()
for player in mod_players:
    data = pandas.Series({"mods": [x.title for x in player.perm_attr if x.id not in UNREFORMED_MODS]}, name=player.name)
    if len(data["mods"]) == 0:
        continue
    mod_list = mod_list.append(data)
mod_list

,mods
Beasley Day,[Minimalist]
J-xo- B-c--ey,[Fire Eater]
Jessica Telephone,"[Superallergic, Honey Roasted]"
Henry Marshallow,[Maximalist]
Doc Anice,[Triple Threat]
Ruslan Greatness,[Friend of Crows]


### Enrich
**Pick a modification for your Team next season.**

This will gives the option of choosing between
* The Fourth Strike: Those with the Fourth Strike will get an extra strike in each at bat.
* High Pressure: This Team will play 25% better in Flooding weather when runners are on base.
* Life of the Party: This team gets 10% more from their Party Time stat boosts.
* Home Field Advantage: Teams with Home Field Advantage will start each home game with one run.
* Affinity for Crows: Players with Affinity for Crows will hit and pitch 50% better during Birds weather.
* Traveling: Traveling teams will play 5% better in Away games.

Seasonal Modifications have been mixed in the past. For Fourth Strike Analysis, see the appropriate
[SIBR Paper](https://research.blaseball-reference.com/Fourth_Strike.pdf).

___
## Blessings

Blessings are selected by raffle across all teams. Each blessing will be awarded to one random team, where teams with
more votes are more likely to be selected.

### Sky's The Limit
**Your entire Team's Rotation will become Maximalists Permanently.**

Maximalists play 250% better during Maximum Blaseball, which is when the bases are loaded with 3 balls, 2 strikes, and 2
outs.

### Afterparty
**Whenever your Team is losing during the First Week of Next Season, they will have a chance of gaining Party Time Boosts.**

This would provide at most 9 days of potential party-time benefits.

### Middling
**Your Team will Overperform during Next Season's Midseason.**

Overperforming will cause our team to play 20% better during the middle 45 games of next season.

### Clutch Up
**Your Cleanup Hitter (the 4th spot in your Lineup) will gain the High Pressure Mod for the following Season.**

High Pressure will cause our 4th place batter (seen below) to play 25% better during Flooding games next the season.

In [484]:
get_stars(pies.lineup[3])

,Batting,Pitching,Baserunning,Defense
J-xo- B-c--ey,6.5,3.5,4.0,3.0


### Absolute Zero
**Oh. Give your Team O Blood.**

This would make all our players have O Blood, and give the team a blood-specific Modification. The effects of this
modification are unknown.

### Water Logged
**Stay Hydrated. Give your Team H2O Blood.**

This would make all our players have H2O Blood, and give the team a blood-specific Modification. The effects of this
modification are unknown.

### Bird Brain
**A Random Player on your Team will gain Affinity For Crows Permanently.**

Affinity for Crows will cause the player to perform 50% better in Bird weather. We... already have Affinity for Crows on
all our players.

### Mulligan
**Your Team's worst hitter will gain the Fourth Strike Permanently.**

This will cause the worst batter (see below) to get called out on 4 strikes instead of 3.

In [485]:
get_stars(worst_batter)

,Batting,Pitching,Baserunning,Defense
-u-e- P-ddles,2.0,2.5,3.0,3.5


### Practice
**Your Team's Lineup and Rotation will have their Defense Boosted from 1% to 4%.**

**Your Team's Lineup will have their Baserunning Boosted from 1% to 4%.**

**Your Team's Rotation will have their Pitching Boosted from 1% to 4%.**

**Your Team's Lineup will have their Hitting Boosted from 1% to 4%.**

These blessings increase the team-wide stats. See below for worst and best case rolls.

#### Batting Practice

**Worst Case**

In [486]:
table, total, avg = improve_team_batting_table(pies, 0.01)
print("Change in Total stars: " + str(total["change_in_batting_stars"]))
print("Change in Average stars: " + str(avg["change_in_batting_stars"]))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.01)), maxVal=1))

Change in Total stars: 0.35551212698283674
Change in Average stars: 0.05078744671183382


,old_batting_stars,new_batting_stars,change_in_batting_stars
Eduardo Woodman,4.165300,4.214065,0.048765
Beasley Day,3.969778,4.019620,0.049842
Huber Frumple,3.806051,3.858926,0.052875
J-xo- B-c--ey,6.318317,6.358075,0.039758
Lang Richardson,3.162214,3.212664,0.050450
-u-e- P-ddles,2.147811,2.215192,0.067381
Jessica Telephone,5.647950,5.694390,0.046441


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.842813,1.029391,0.856540,0.660666,0.585519,0.585517,0.584554,0.090000,0.715869
Beasley Day,0.803924,0.984873,0.860851,0.310545,0.511918,0.262384,0.696625,0.010000,0.975368
Huber Frumple,0.771785,0.433284,1.179818,0.887497,1.207390,0.335580,0.654357,0.044720,1.073890
J-xo- B-c--ey,1.271615,1.503269,0.997449,1.686092,1.862606,0.010000,1.942640,0.010000,1.337099
Lang Richardson,0.642533,0.822722,0.410621,0.632843,0.951821,0.195246,0.508400,0.010000,0.561015
-u-e- P-ddles,0.443038,1.043698,0.194577,0.685196,0.161908,0.620778,0.120821,0.047271,1.020015
Jessica Telephone,1.138878,1.123282,1.226455,1.336294,0.992475,0.205902,1.584396,0.090000,0.929039


**Best Case**

In [487]:
table, total, avg = improve_team_batting_table(pies, 0.04)
print("Change in Total stars: " + str(total["change_in_batting_stars"]))
print("Change in Average stars: " + str(avg["change_in_batting_stars"]))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.04)), maxVal=1))


Change in Total stars: 1.4232257241707749
Change in Average stars: 0.203317960595825


,old_batting_stars,new_batting_stars,change_in_batting_stars
Eduardo Woodman,4.165300,4.359810,0.194510
Beasley Day,3.969778,4.169113,0.199334
Huber Frumple,3.806051,4.016283,0.210232
J-xo- B-c--ey,6.318317,6.487264,0.168947
Lang Richardson,3.162214,3.363875,0.201662
-u-e- P-ddles,2.147811,2.410803,0.262992
Jessica Telephone,5.647950,5.833499,0.185549


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.871962,1.059391,0.886540,0.690666,0.615519,0.555517,0.614554,0.060000,0.745869
Beasley Day,0.833823,1.014873,0.890851,0.340545,0.541918,0.232384,0.726625,0.010000,1.005368
Huber Frumple,0.803257,0.463284,1.209818,0.917497,1.237390,0.305580,0.684357,0.014720,1.103890
J-xo- B-c--ey,1.297453,1.533269,1.027449,1.716092,1.892606,0.010000,1.972640,0.010000,1.367099
Lang Richardson,0.672775,0.852722,0.440621,0.662843,0.981821,0.165246,0.538400,0.010000,0.591015
-u-e- P-ddles,0.482161,1.073698,0.224577,0.715196,0.191908,0.590778,0.150821,0.017271,1.050015
Jessica Telephone,1.166700,1.153282,1.256455,1.366294,1.022475,0.175902,1.614396,0.060000,0.959039


#### Pitching Practice

**Worst Case**

In [488]:
table, total, avg = improve_team_pitching_table(pies, 0.01)
print("Change in Total stars: " + str(total["change_in_pitching_stars"]))
print("Change in Average stars: " + str(avg["change_in_pitching_stars"]))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.01)), maxVal=1))

Change in Total stars: 0.30617389611087464
Change in Average stars: 0.06123477922217493


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,3.439741,3.502125,0.062384
Nerd Pacheco,4.180600,4.246417,0.065817
Doc Anice,2.841172,2.900636,0.059464
Ruslan Greatness,4.096695,4.154610,0.057914
Elvis Figueroa,5.483164,5.543758,0.060594


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.700425,0.433292,1.116023,1.391125,0.784138,0.755639,0.831668
Nerd Pacheco,0.849283,1.097720,1.076351,0.282113,0.618134,0.942212,0.494500
Doc Anice,0.580127,0.938114,0.369045,0.846037,0.466020,0.608965,0.154698
Ruslan Greatness,0.830922,0.780870,0.796481,1.279963,1.036832,0.639742,1.430762
Elvis Figueroa,1.108752,1.169675,1.146143,1.082682,0.796089,0.471405,0.731203


**Best Case**

In [489]:
table, total, avg = improve_team_pitching_table(pies, 0.04)
print("Change in Total stars: " + str(total["change_in_pitching_stars"]))
print("Change in Average stars: " + str(avg["change_in_pitching_stars"]))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.04)), maxVal=1))

Change in Total stars: 1.224225960817773
Change in Average stars: 0.2448451921635546


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,3.439741,3.689096,0.249355
Nerd Pacheco,4.180600,4.442577,0.261976
Doc Anice,2.841172,3.078983,0.237811
Ruslan Greatness,4.096695,4.328975,0.232279
Elvis Figueroa,5.483164,5.725968,0.242804


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.737819,0.463292,1.146023,1.421125,0.814138,0.785639,0.861668
Nerd Pacheco,0.888515,1.127720,1.106351,0.312113,0.648134,0.972212,0.524500
Doc Anice,0.615797,0.968114,0.399045,0.876037,0.496020,0.638965,0.184698
Ruslan Greatness,0.865795,0.810870,0.826481,1.309963,1.066832,0.669742,1.460762
Elvis Figueroa,1.145194,1.199675,1.176143,1.112682,0.826089,0.501405,0.761203


#### Running Practice

**Worst Case**

In [490]:
table, total, avg = improve_team_baserunning_table(pies, 0.01)
print("Change in Total stars: " + str(total["change_in_baserunning_stars"]))
print("Change in Average stars: " + str(avg["change_in_baserunning_stars"]))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(pies, 0.01)), maxVal=1))

Change in Total stars: 0.33651171073021935
Change in Average stars: 0.048073101532888476


,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Eduardo Woodman,3.519368,3.566968,0.047600
Beasley Day,3.368105,3.416292,0.048187
Huber Frumple,4.481955,4.527626,0.045671
J-xo- B-c--ey,4.178573,4.225306,0.046732
Lang Richardson,3.581538,3.629955,0.048416
-u-e- P-ddles,2.817447,2.869284,0.051836
Jessica Telephone,3.755559,3.803628,0.048069


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Eduardo Woodman,0.713394,0.627191,1.050482,0.536095,0.717616,0.870506
Beasley Day,0.683258,0.576884,0.623076,1.144900,0.553849,0.878432
Huber Frumple,0.905525,0.852208,0.908349,0.962964,1.094902,0.861072
J-xo- B-c--ey,0.845061,0.852412,1.332573,0.671710,0.663193,0.695217
Lang Richardson,0.725991,0.752498,0.431602,0.855647,0.456945,0.998950
-u-e- P-ddles,0.573857,0.617851,0.240396,1.006456,0.465636,0.381812
Jessica Telephone,0.760726,0.588318,0.985123,1.093708,0.779102,1.097069


**Best Case**

In [491]:
table, total, avg = improve_team_baserunning_table(pies, 0.04)
print("Change in Total stars: " + str(total["change_in_baserunning_stars"]))
print("Change in Average stars: " + str(avg["change_in_baserunning_stars"]))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(pies, 0.04)), maxVal=1))

Change in Total stars: 1.3409830407026226
Change in Average stars: 0.19156900581466038


,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Eduardo Woodman,3.519368,3.709222,0.189854
Beasley Day,3.368105,3.560234,0.192129
Huber Frumple,4.481955,4.664316,0.182361
J-xo- B-c--ey,4.178573,4.365055,0.186482
Lang Richardson,3.581538,3.774434,0.192895
-u-e- P-ddles,2.817447,3.023073,0.205626
Jessica Telephone,3.755559,3.947195,0.191635


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Eduardo Woodman,0.741844,0.657191,1.080482,0.566095,0.747616,0.900506
Beasley Day,0.712047,0.606884,0.653076,1.174900,0.583849,0.908432
Huber Frumple,0.932863,0.882208,0.938349,0.992964,1.124902,0.891072
J-xo- B-c--ey,0.873011,0.882412,1.362573,0.701710,0.693193,0.725217
Lang Richardson,0.754887,0.782498,0.461602,0.885647,0.486945,1.028950
-u-e- P-ddles,0.604615,0.647851,0.270396,1.036456,0.495636,0.411812
Jessica Telephone,0.789439,0.618318,1.015123,1.123708,0.809102,1.127069


#### Defense Practice

**Worst Case**

In [492]:
table, total, avg = improve_team_defense_table(pies, 0.01)
print("Change in Total stars: " + str(total["change_in_defense_stars"]))
print("Change in Average stars: " + str(avg["change_in_defense_stars"]))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(pies, 0.01)), maxVal=1))

Change in Total stars: 0.5184225912056011
Change in Average stars: 0.04320188260046676


,old_defense_stars,new_defense_stars,change_in_defense_stars
Eduardo Woodman,3.554369,3.597999,0.043631
Beasley Day,2.720315,2.769795,0.049480
Huber Frumple,3.430757,3.473507,0.042750
J-xo- B-c--ey,2.991285,3.036725,0.045440
Lang Richardson,3.744672,3.788714,0.044042
-u-e- P-ddles,3.596398,3.640719,0.044321
Jessica Telephone,3.145093,3.189159,0.044066
Henry Marshallow,4.436965,4.476794,0.039829
Nerd Pacheco,2.802547,2.853097,0.050550
Doc Anice,3.683109,3.724647,0.041538


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Eduardo Woodman,0.719600,0.449614,0.434310,0.924677,1.024311,1.030885
Beasley Day,0.553959,0.295287,0.347933,1.243318,0.390529,0.530959
Huber Frumple,0.694701,0.571542,0.441824,0.528990,1.150583,0.674561
J-xo- B-c--ey,0.607345,0.594073,0.405521,0.374186,0.878701,0.357866
Lang Richardson,0.757743,1.053616,0.950524,0.384060,0.400675,0.404329
-u-e- P-ddles,0.728144,0.499394,0.982955,0.287902,0.664868,0.908316
Jessica Telephone,0.637832,0.620429,0.355718,0.566907,0.681559,0.592176
Henry Marshallow,0.895359,0.828329,1.257928,0.719309,0.392427,1.080387
Nerd Pacheco,0.570619,0.283129,0.706025,0.816231,0.222686,0.503905
Doc Anice,0.744929,0.673859,0.838026,0.831993,0.373779,0.530595


**Best Case**

In [493]:
table, total, avg = improve_team_defense_table(pies, 0.04)
print("Change in Total stars: " + str(total["change_in_defense_stars"]))
print("Change in Average stars: " + str(avg["change_in_defense_stars"]))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(pies, 0.04)), maxVal=1))

Change in Total stars: 2.0501748011089695
Change in Average stars: 0.17084790009241413


,old_defense_stars,new_defense_stars,change_in_defense_stars
Eduardo Woodman,3.554369,3.726999,0.172630
Beasley Day,2.720315,2.915229,0.194914
Huber Frumple,3.430757,3.600155,0.169398
J-xo- B-c--ey,2.991285,3.170934,0.179649
Lang Richardson,3.744672,3.918681,0.174010
-u-e- P-ddles,3.596398,3.771363,0.174965
Jessica Telephone,3.145093,3.319457,0.174364
Henry Marshallow,4.436965,4.594812,0.157848
Nerd Pacheco,2.802547,3.000867,0.198320
Doc Anice,3.683109,3.847719,0.164610


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Eduardo Woodman,0.745400,0.479614,0.464310,0.954677,1.054311,1.060885
Beasley Day,0.583046,0.325287,0.377933,1.273318,0.420529,0.560959
Huber Frumple,0.720031,0.601542,0.471824,0.558990,1.180583,0.704561
J-xo- B-c--ey,0.634187,0.624073,0.435521,0.404186,0.908701,0.387866
Lang Richardson,0.783736,1.083616,0.980524,0.414060,0.430675,0.434329
-u-e- P-ddles,0.754273,0.529394,1.012955,0.317902,0.694868,0.938316
Jessica Telephone,0.663891,0.650429,0.385718,0.596907,0.711559,0.622176
Henry Marshallow,0.918962,0.858329,1.287928,0.749309,0.422427,1.110387
Nerd Pacheco,0.600173,0.313129,0.736025,0.846231,0.252686,0.533905
Doc Anice,0.769544,0.703859,0.868026,0.861993,0.403779,0.560595


### Shadow Practice
**Your Team's Shadows will have their Defense/Baserunning/Pitching/Hitting Boosted from 1% to 12%.**

These blessings increase the team-wide stats or our shadow players. See below for worst and best case rolls. This
assumes that it will only boost the respective shadow positions, so batting will only affect the bench, etc.

#### Shadow Batting Practice

**Worst Case**

In [494]:
table, total, avg = improve_team_batting_table(pies, 0.01, shadows=True)
print("Change in Total stars: " + str(total["change_in_batting_stars"]))
print("Change in Average stars: " + str(avg["change_in_batting_stars"]))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.01, shadows=True)), maxVal=1))

Change in Total stars: 0.465582514784211
Change in Average stars: 0.0775970857973685


,old_batting_stars,new_batting_stars,change_in_batting_stars
Adrian Melon,2.353291,2.414010,0.060718
August Obrien,3.408347,3.470699,0.062352
Mcfarland Vargas,1.383943,1.476773,0.092829
Twofurious Puddles,2.635533,2.722015,0.086482
Mummy Cantburn,1.390958,1.477752,0.086794
Doginic Burgertoes,2.079285,2.155693,0.076407


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Adrian Melon,0.482802,0.312171,0.718122,0.961649,0.112259,0.265678,0.342170,0.090000,0.553764
August Obrien,0.694140,0.651217,0.937569,0.923891,0.502708,0.913854,0.566849,0.090000,0.473958
Mcfarland Vargas,0.295355,0.127084,0.786150,0.317714,0.185366,0.974279,0.427832,0.090000,0.561343
Twofurious Puddles,0.544403,0.587625,0.490623,0.867061,0.986273,0.802102,0.019182,0.090000,0.484513
Mummy Cantburn,0.295550,0.105592,0.586609,0.957265,0.535890,0.906564,0.022553,0.090000,0.937160
Doginic Burgertoes,0.431139,0.186724,0.994426,0.115629,0.944521,0.202919,0.048098,0.751237,0.314905


**Best Case**

In [495]:
table, total, avg = improve_team_batting_table(pies, 0.12, shadows=True)
print("Change in Total stars: " + str(total["change_in_batting_stars"]))
print("Change in Average stars: " + str(avg["change_in_batting_stars"]))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.12, shadows=True)), maxVal=1))


Change in Total stars: 4.548014966625795
Change in Average stars: 0.7580024944376325


,old_batting_stars,new_batting_stars,change_in_batting_stars
Adrian Melon,2.353291,3.033053,0.679762
August Obrien,3.408347,4.082129,0.673782
Mcfarland Vargas,1.383943,2.225020,0.841077
Twofurious Puddles,2.635533,3.354184,0.718652
Mummy Cantburn,1.390958,2.220729,0.829771
Doginic Burgertoes,2.079285,2.884257,0.804971


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Adrian Melon,0.606611,0.422171,0.828122,1.071649,0.222259,0.155678,0.452170,0.010000,0.663764
August Obrien,0.816426,0.761217,1.047569,1.033891,0.612708,0.803854,0.676849,0.010000,0.583958
Mcfarland Vargas,0.445004,0.237084,0.896150,0.427714,0.295366,0.864279,0.537832,0.010000,0.671343
Twofurious Puddles,0.670837,0.697625,0.600623,0.977061,1.096273,0.692102,0.129182,0.010000,0.594513
Mummy Cantburn,0.444146,0.215592,0.696609,1.067265,0.645890,0.796564,0.132553,0.010000,1.047160
Doginic Burgertoes,0.576851,0.296724,1.104426,0.225629,1.054521,0.092919,0.158098,0.641237,0.424905


#### Shadow Pitching Practice

**Worst Case**

In [496]:
table, total, avg = improve_team_pitching_table(pies, 0.01, shadows=True)
print("Change in Total stars: " + str(total["change_in_pitching_stars"]))
print("Change in Average stars: " + str(avg["change_in_pitching_stars"]))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.01, shadows=True)), maxVal=1))

Change in Total stars: 0.6959431944569421
Change in Average stars: 0.0773270216063269


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Castillo Logan,3.022344,3.093824,0.071480
Bambi Perez,0.340030,0.431085,0.091055
Jesús Rodriguez,1.281753,1.337202,0.055449
Jefferson Delacruz,0.583451,0.684939,0.101488
Case Lancaster,3.291826,3.357347,0.065521
Haruta Byrd,2.728771,2.789924,0.061153
Javier Lotus,0.765838,0.818349,0.052510
Tiana Takahashi,3.094250,3.226215,0.131965
Lucy Tokkan,3.708740,3.774062,0.065322


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Castillo Logan,0.618765,0.972056,0.689044,0.176984,1.004242,0.100067,0.240832
Bambi Perez,0.086217,0.028422,0.488632,0.284202,0.161506,0.592147,0.120881
Jesús Rodriguez,0.267440,0.201237,0.689646,0.294989,0.196662,0.401038,0.818921
Jefferson Delacruz,0.136988,0.215741,0.195705,0.396775,0.014881,0.693663,0.107056
Case Lancaster,0.671469,0.877044,0.839197,0.200119,0.835333,0.878251,0.092117
Haruta Byrd,0.557985,0.756009,0.694712,0.199109,0.612043,0.765252,0.370515
Javier Lotus,0.163670,0.117195,0.280303,0.280723,0.808413,0.499265,0.631886
Tiana Takahashi,0.645243,0.920065,0.971592,0.726677,0.038954,0.606122,0.955940
Lucy Tokkan,0.754812,0.911966,1.005562,0.273758,0.983020,0.190980,0.331805


**Best Case**

In [497]:
table, total, avg = improve_team_pitching_table(pies, 0.12, shadows=True)
print("Change in Total stars: " + str(total["change_in_pitching_stars"]))
print("Change in Average stars: " + str(avg["change_in_pitching_stars"]))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.12, shadows=True)), maxVal=1))

Change in Total stars: 6.961428824088222
Change in Average stars: 0.773492091565358


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Castillo Logan,3.022344,3.833175,0.810831
Bambi Perez,0.340030,1.146391,0.806361
Jesús Rodriguez,1.281753,1.946115,0.664362
Jefferson Delacruz,0.583451,1.295765,0.712314
Case Lancaster,3.291826,4.057331,0.765505
Haruta Byrd,2.728771,3.450752,0.721981
Javier Lotus,0.765838,1.391343,0.625504
Tiana Takahashi,3.094250,4.179182,1.084931
Lucy Tokkan,3.708740,4.478378,0.769638


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Castillo Logan,0.766635,1.082056,0.799044,0.286984,1.114242,0.210067,0.350832
Bambi Perez,0.229278,0.138422,0.598632,0.394202,0.271506,0.702147,0.230881
Jesús Rodriguez,0.389223,0.311237,0.799646,0.404989,0.306662,0.511038,0.928921
Jefferson Delacruz,0.259153,0.325741,0.305705,0.506775,0.124881,0.803663,0.217056
Case Lancaster,0.811466,0.987044,0.949197,0.310119,0.945333,0.988251,0.202117
Haruta Byrd,0.690150,0.866009,0.804712,0.309109,0.722043,0.875252,0.480515
Javier Lotus,0.278269,0.227195,0.390303,0.390723,0.918413,0.609265,0.741886
Tiana Takahashi,0.835836,1.030065,1.081592,0.836677,0.148954,0.716122,1.065940
Lucy Tokkan,0.895676,1.021966,1.115562,0.383758,1.093020,0.300980,0.441805


#### Shadow Running Practice

**Worst Case**

In [498]:
table, total, avg = improve_team_baserunning_table(pies, 0.01, shadows=True)
print("Change in Total stars: " + str(total["change_in_baserunning_stars"]))
print("Change in Average stars: " + str(avg["change_in_baserunning_stars"]))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(pies, 0.01, shadows=True)), maxVal=1))

Change in Total stars: 0.3779591112420766
Change in Average stars: 0.06299318520701276


,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Adrian Melon,3.116697,3.171746,0.055049
August Obrien,3.318856,3.378726,0.059870
Mcfarland Vargas,3.783245,3.831944,0.048700
Twofurious Puddles,1.817993,1.877135,0.059142
Mummy Cantburn,2.021513,2.083616,0.062102
Doginic Burgertoes,1.779564,1.872660,0.093096


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Adrian Melon,0.634349,0.850831,0.594640,0.495051,0.186399,0.431241
August Obrien,0.675745,0.976470,0.150389,0.711812,0.491149,0.425344
Mcfarland Vargas,0.766389,0.932765,0.567085,0.822168,0.492377,0.431246
Twofurious Puddles,0.375427,0.203390,0.388033,0.524860,0.898752,0.873079
Mummy Cantburn,0.416723,0.298250,0.842320,0.949393,0.108762,0.769435
Doginic Burgertoes,0.374532,0.332288,0.037095,0.627854,0.637448,0.903013


**Best Case**

In [499]:
table, total, avg = improve_team_baserunning_table(pies, 0.12, shadows=True)
print("Change in Total stars: " + str(total["change_in_baserunning_stars"]))
print("Change in Average stars: " + str(avg["change_in_baserunning_stars"]))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(pies, 0.12, shadows=True)), maxVal=1))

Change in Total stars: 4.0516944755187625
Change in Average stars: 0.6752824125864604


,old_baserunning_stars,new_baserunning_stars,change_in_baserunning_stars
Adrian Melon,3.116697,3.747971,0.631273
August Obrien,3.318856,3.988228,0.669372
Mcfarland Vargas,3.783245,4.359164,0.575919
Twofurious Puddles,1.817993,2.492548,0.674555
Mummy Cantburn,2.021513,2.711657,0.690144
Doginic Burgertoes,1.779564,2.589995,0.810431


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Adrian Melon,0.749594,0.960831,0.704640,0.605051,0.296399,0.541241
August Obrien,0.797646,1.086470,0.260389,0.821812,0.601149,0.535344
Mcfarland Vargas,0.871833,1.042765,0.677085,0.932168,0.602377,0.541246
Twofurious Puddles,0.498510,0.313390,0.498033,0.634860,1.008752,0.983079
Mummy Cantburn,0.542331,0.408250,0.952320,1.059393,0.218762,0.879435
Doginic Burgertoes,0.517999,0.442288,0.147095,0.737854,0.747448,1.013013


#### Shadow Defense Practice

**Worst Case**

In [500]:
table, total, avg = improve_team_defense_table(pies, 0.01, shadows=True)
print("Change in Total stars: " + str(total["change_in_defense_stars"]))
print("Change in Average stars: " + str(avg["change_in_defense_stars"]))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(pies, 0.01, shadows=True)), maxVal=1))

Change in Total stars: 1.1356251052662554
Change in Average stars: 0.07570834035108369


,old_defense_stars,new_defense_stars,change_in_defense_stars
Adrian Melon,2.664412,2.718599,0.054187
August Obrien,2.035102,2.125446,0.090344
Mcfarland Vargas,2.071032,2.162907,0.091874
Twofurious Puddles,3.610790,3.655879,0.045089
Mummy Cantburn,3.289650,3.332702,0.043052
Doginic Burgertoes,1.538638,1.636112,0.097474
Castillo Logan,2.996505,3.049184,0.052679
Bambi Perez,2.859667,2.923939,0.064272
Jesús Rodriguez,1.417111,1.614089,0.196978
Jefferson Delacruz,3.914262,3.955411,0.041149


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Adrian Melon,0.543720,0.198840,0.353662,0.915908,0.526713,0.946550
August Obrien,0.425089,0.064615,0.493368,0.847103,0.337928,0.662261
Mcfarland Vargas,0.432581,0.064401,0.366182,0.600229,0.751915,0.914134
Twofurious Puddles,0.731176,0.857265,0.919861,0.270410,0.665028,0.390553
Mummy Cantburn,0.666540,0.601982,0.406224,0.526128,0.635545,0.865624
Doginic Burgertoes,0.327222,0.057259,0.874391,0.425999,0.128550,0.102532
Castillo Logan,0.609837,0.759614,0.201755,0.727648,0.501621,0.829861
Bambi Perez,0.584788,0.732095,0.807496,0.744400,0.151465,0.118700
Jesús Rodriguez,0.322818,0.277421,0.105104,0.971620,0.944389,0.015755
Jefferson Delacruz,0.791082,0.986397,0.431596,0.814808,0.794419,0.818188


**Best Case**

In [501]:
table, total, avg = improve_team_defense_table(pies, 0.12, shadows=True)
print("Change in Total stars: " + str(total["change_in_defense_stars"]))
print("Change in Average stars: " + str(avg["change_in_defense_stars"]))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(pies, 0.12, shadows=True)), maxVal=1))

Change in Total stars: 10.166516472113571
Change in Average stars: 0.6777677648075714


,old_defense_stars,new_defense_stars,change_in_defense_stars
Adrian Melon,2.664412,3.266263,0.601851
August Obrien,2.035102,2.854276,0.819174
Mcfarland Vargas,2.071032,2.901058,0.830026
Twofurious Puddles,3.610790,4.126072,0.515282
Mummy Cantburn,3.289650,3.789322,0.499672
Doginic Burgertoes,1.538638,2.430513,0.891874
Castillo Logan,2.996505,3.579799,0.583295
Bambi Perez,2.859667,3.526931,0.667264
Jesús Rodriguez,1.417111,2.504935,1.087824
Jefferson Delacruz,3.914262,4.392425,0.478163


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Adrian Melon,0.653253,0.308840,0.463662,1.025908,0.636713,1.056550
August Obrien,0.570855,0.174615,0.603368,0.957103,0.447928,0.772261
Mcfarland Vargas,0.580212,0.174401,0.476182,0.710229,0.861915,1.024134
Twofurious Puddles,0.825214,0.967265,1.029861,0.380410,0.775028,0.500553
Mummy Cantburn,0.757864,0.711982,0.516224,0.636128,0.745545,0.975624
Doginic Burgertoes,0.486103,0.167259,0.984391,0.535999,0.238550,0.212532
Castillo Logan,0.715960,0.869614,0.311755,0.837648,0.611621,0.939861
Bambi Perez,0.705386,0.842095,0.917496,0.854400,0.261465,0.228700
Jesús Rodriguez,0.500987,0.387421,0.215104,1.081620,1.054389,0.125755
Jefferson Delacruz,0.878485,1.096397,0.541596,0.924808,0.904419,0.928188


### Easy Out
**Move the worst Hitter on your Team to the last spot in your Lineup.**

See below for the new lineup order.

In [502]:
new_lineup = pies.lineup.copy()
new_lineup.append(new_lineup.pop(new_lineup.index(worst_batter)))
get_stars(new_lineup)

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,4.0,2.0,3.5,3.5
Beasley Day,4.0,2.5,3.5,2.5
Huber Frumple,4.0,2.5,4.5,3.5
J-xo- B-c--ey,6.5,3.5,4.0,3.0
Lang Richardson,3.0,0.5,3.5,3.5
Jessica Telephone,5.5,2.5,4.0,3.0
-u-e- P-ddles,2.0,2.5,3.0,3.5


### Lead Off
**Move the best Baserunner on your Team to the first position in your Lineup.**

See below for the new lineup order.

In [503]:
new_lineup.sort(key=lambda x: x.baserunning_rating, reverse=True)
best_baserunner = new_lineup[0]
new_lineup = pies.lineup.copy()
new_lineup.insert(0, new_lineup.pop(new_lineup.index(best_baserunner)))
get_stars(new_lineup)

,Batting,Pitching,Baserunning,Defense
Huber Frumple,4.0,2.5,4.5,3.5
Eduardo Woodman,4.0,2.0,3.5,3.5
Beasley Day,4.0,2.5,3.5,2.5
J-xo- B-c--ey,6.5,3.5,4.0,3.0
Lang Richardson,3.0,0.5,3.5,3.5
-u-e- P-ddles,2.0,2.5,3.0,3.5
Jessica Telephone,5.5,2.5,4.0,3.0


### Blood Pact
**Recruit a Random Player with the Siphon Modification to your Team.**

See below for list of all living players with Siphon. If this refers to generating a new player instead of stealing an
existing one then it is the same as a reroll, which is likely worse than our team average.

In [504]:
all_players = Player.load_all()
siphons = [x for x in all_players.values() if x.league_team_id is not None and x.deceased is False and "SIPHON" in x._perm_attr_ids]
get_stars(siphons)

,Batting,Pitching,Baserunning,Defense
Halexandrey Walton,3.0,1.0,1.5,4.0
Glabe Moon,3.5,2.5,5.0,6.0
Silvia Rugrat,2.5,3.0,1.0,2.0
Spears Taylor,1.5,1.0,1.5,5.5
Cory Ross,4.0,3.0,5.0,7.0
Dudley Mueller,5.0,3.0,3.5,4.5
Jordan Hildebert,4.5,3.0,3.5,3.5
Randy Dennis,4.0,1.5,2.5,3.5
Isaac Johnson,3.0,2.0,3.5,6.5
Logan Rodriguez,2.0,2.5,3.0,4.5


### Fringe Benefit
**Recruit a Random Player with the Perk Modification to your Team.**

See below for list of all living players with Perk. If this refers to generating a new player instead of stealing an
existing one then it is the same as a reroll, which is likely worse than our team average.

In [505]:
all_players = Player.load_all()
siphons = [x for x in all_players.values() if x.league_team_id is not None and x.deceased is False and "PERK" in x._perm_attr_ids]
get_stars(siphons)

,Batting,Pitching,Baserunning,Defense
Halexandrey Walton,3.0,1.0,1.5,4.0
Cornelius Games,2.5,3.5,2.5,4.0
Glabe Moon,3.5,2.5,5.0,6.0
Elijah Valenzuela,1.0,1.0,2.5,2.5
Ren Morin,1.5,1.0,1.5,1.5
Baby Doyle,5.0,1.5,2.5,4.0
Parker Meng,2.0,4.0,2.5,3.5
Sandie Turner,4.5,4.5,4.5,3.5
Holden Stanton,3.0,0.5,2.5,4.0
Commissioner Vapor,4.5,2.5,4.0,3.0


### Epiphany
**A Random Player from your Team's Lineup will have their Divinity Boosted from -20% to 60%.**

Divinity is the player attribute associated with hitting Home Runs. See below for how we compare with the rest of the
league and stat change in the best and worst case rolls.

**League Average**

In [506]:
x = pandas.DataFrame([pandas.Series({"Divinity": mean([y.divinity for y in x.lineup])}, name=x.nickname) for x in all_teams.values()]).sort_values("Divinity", ascending=False)
set_heatmap(x, maxVal=1)

,Divinity
Crabs,1.012352
Flowers,0.947467
Shoe Thieves,0.886002
Steaks,0.829129
Pies,0.808044
Millennials,0.792098
Moist Talkers,0.785299
Worms,0.768754
Spies,0.761007
Dale,0.758666


**Best Case**


In [507]:
new_team = []
for player in pies.lineup:
    new_team.append(player.simulated_copy(buffs={"divinity": 0.6}))
set_heatmap(get_batting_stlats(new_team), maxVal=1)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.004883,1.019391,1.446540,0.650666,0.575519,0.595517,0.574554,0.100000,0.705869
Beasley Day,0.957008,0.974873,1.450851,0.300545,0.501918,0.272384,0.686625,0.001000,0.965368
Huber Frumple,0.879910,0.423284,1.769818,0.877497,1.197390,0.345580,0.644357,0.054720,1.063890
J-xo- B-c--ey,1.492094,1.493269,1.587449,1.676092,1.852606,0.001000,1.932640,0.001000,1.327099
Lang Richardson,0.871282,0.812722,1.000621,0.622843,0.941821,0.205246,0.498400,0.001000,0.551015
-u-e- P-ddles,0.712831,1.033698,0.784577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015
Jessica Telephone,1.299768,1.113282,1.816455,1.326294,0.982475,0.215902,1.574396,0.100000,0.919039


**Worst Case**

In [508]:
new_team = []
for player in pies.lineup:
    new_team.append(player.simulated_copy(buffs={"divinity": -0.2}))
set_heatmap(get_batting_stlats(new_team), maxVal=1)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.758068,1.019391,0.646540,0.650666,0.575519,0.595517,0.574554,0.100000,0.705869
Beasley Day,0.722880,0.974873,0.650851,0.300545,0.501918,0.272384,0.686625,0.001000,0.965368
Huber Frumple,0.712861,0.423284,0.969818,0.877497,1.197390,0.345580,0.644357,0.054720,1.063890
J-xo- B-c--ey,1.167425,1.493269,0.787449,1.676092,1.852606,0.001000,1.932640,0.001000,1.327099
Lang Richardson,0.496473,0.812722,0.200621,0.622843,0.941821,0.205246,0.498400,0.001000,0.551015
-u-e- P-ddles,0.154833,1.033698,0.010000,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015
Jessica Telephone,1.060762,1.113282,1.016455,1.326294,0.982475,0.215902,1.574396,0.100000,0.919039


### Dirty Bulk
**A Random Player from your Team's Lineup will have their Musclitude Boosted from -20% to 60%.**

Musclitude is the player attribute associated with hitting doubles. See below for how we compare with the rest of the
league and stat change in the best and worst case rolls.

**League Average**

In [509]:
x = pandas.DataFrame([pandas.Series({"Musclitude": mean([y.musclitude for y in x.lineup])}, name=x.nickname) for x in all_teams.values()]).sort_values("Musclitude", ascending=False)
set_heatmap(x, maxVal=1)

,Musclitude
Shoe Thieves,0.940478
Fridays,0.917716
Crabs,0.912459
Pies,0.875590
Worms,0.870043
Millennials,0.862589
Moist Talkers,0.830861
Firefighters,0.825014
Georgias,0.743021
Steaks,0.741111


**Best Case**

In [510]:
new_team = []
for player in pies.lineup:
    new_team.append(player.simulated_copy(buffs={"musclitude": 0.6}))
set_heatmap(get_batting_stlats(new_team), maxVal=1)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.874903,1.019391,0.846540,1.250666,0.575519,0.595517,0.574554,0.100000,0.705869
Beasley Day,0.862067,0.974873,0.850851,0.900545,0.501918,0.272384,0.686625,0.001000,0.965368
Huber Frumple,0.791545,0.423284,1.169818,1.477497,1.197390,0.345580,0.644357,0.054720,1.063890
J-xo- B-c--ey,1.292999,1.493269,0.987449,2.276092,1.852606,0.001000,1.932640,0.001000,1.327099
Lang Richardson,0.665266,0.812722,0.400621,1.222843,0.941821,0.205246,0.498400,0.001000,0.551015
-u-e- P-ddles,0.450544,1.033698,0.184577,1.275196,0.151908,0.630778,0.110821,0.057271,1.010015
Jessica Telephone,1.161655,1.113282,1.216455,1.926294,0.982475,0.215902,1.574396,0.100000,0.919039


**Worst Case**

In [511]:
new_team = []
for player in pies.lineup:
    new_team.append(player.simulated_copy(buffs={"musclitude": -0.2}))
set_heatmap(get_batting_stlats(new_team), maxVal=1)

,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.810426,1.019391,0.846540,0.450666,0.575519,0.595517,0.574554,0.100000,0.705869
Beasley Day,0.731358,0.974873,0.850851,0.100545,0.501918,0.272384,0.686625,0.001000,0.965368
Huber Frumple,0.746585,0.423284,1.169818,0.677497,1.197390,0.345580,0.644357,0.054720,1.063890
J-xo- B-c--ey,1.251678,1.493269,0.987449,1.476092,1.852606,0.001000,1.932640,0.001000,1.327099
Lang Richardson,0.614336,0.812722,0.400621,0.422843,0.941821,0.205246,0.498400,0.001000,0.551015
-u-e- P-ddles,0.418393,1.033698,0.184577,0.475196,0.151908,0.630778,0.110821,0.057271,1.010015
Jessica Telephone,1.115827,1.113282,1.216455,1.126294,0.982475,0.215902,1.574396,0.100000,0.919039


### Living Single
**A Random Player from your Team's Rotation will have their Overpowerment Boosted by 40%.**

Overpowerment is the player attribute associated with reducing opposing team doubles, triples, and home runs. See below
for how we compare with the rest of the league and for the individual stat change.

**League Average**

In [512]:
x = pandas.DataFrame([pandas.Series({"Overpowerment": mean([y.overpowerment for y in x.rotation])}, name=x.nickname) for x in all_teams.values()]).sort_values("Overpowerment", ascending=False)
set_heatmap(x, maxVal=1)

,Overpowerment
Shoe Thieves,0.999123
Pies,0.966384
Breath Mints,0.957291
Dale,0.947354
Tacos,0.817590
Millennials,0.808205
Magic,0.795096
Fridays,0.767351
Georgias,0.766889
Worms,0.756569


In [513]:
new_team = []
for player in pies.rotation:
    new_team.append(player.simulated_copy(buffs={"overpowerment": 0.4}))
set_heatmap(get_pitching_stlats(new_team), maxVal=1)

,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.714702,0.423292,1.106023,1.781125,0.774138,0.745639,0.821668
Nerd Pacheco,0.957575,1.087720,1.066351,0.672113,0.608134,0.932212,0.484500
Doc Anice,0.602557,0.928114,0.359045,1.236037,0.456020,0.598965,0.144698
Ruslan Greatness,0.853692,0.770870,0.786481,1.669963,1.026832,0.629742,1.420762
Elvis Figueroa,1.150024,1.159675,1.136143,1.472682,0.786089,0.461405,0.721203


### Triple Crown
**A Random Player from your Team's Lineup will have their Ground Friction Boosted by 40%.**

Ground Friction is the player attribute associated with hitting Triples. See below for how we compare with the rest of
the league and for the individual stat change.

**League Average**

In [514]:
x = pandas.DataFrame([pandas.Series({"Ground Friction": mean([y.ground_friction for y in x.lineup])}, name=x.nickname) for x in all_teams.values()]).sort_values("Ground Friction", ascending=False)
set_heatmap(x, maxVal=1)

,Ground Friction
Crabs,0.850229
Pies,0.816151
Worms,0.813834
Tacos,0.809953
Shoe Thieves,0.801134
Steaks,0.785954
Magic,0.739652
Breath Mints,0.712242
Georgias,0.665500
Garages,0.652149


In [515]:
new_team = []
for player in pies.lineup:
    new_team.append(player.simulated_copy(buffs={"groundFriction": 0.4}))
set_heatmap(get_baserunning_stlats(new_team), maxVal=1)

,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Eduardo Woodman,0.731263,0.617191,1.040482,0.526095,0.707616,1.260506
Beasley Day,0.699630,0.566884,0.613076,1.134900,0.543849,1.268432
Huber Frumple,0.931599,0.842208,0.898349,0.952964,1.084902,1.251072
J-xo- B-c--ey,0.875038,0.842412,1.322573,0.661710,0.653193,1.085217
Lang Richardson,0.741056,0.742498,0.421602,0.845647,0.446945,1.388950
-u-e- P-ddles,0.606184,0.607851,0.230396,0.996456,0.455636,0.771812
Jessica Telephone,0.775018,0.578318,0.975123,1.083708,0.769102,1.487069


---
## Appendix
* [Description of Attributes](https://www.blaseball.wiki/w/Player_Attributes)
* [Stlat Viewer](https://slavfox.space/abslve/?foreboding-kaleidoscope#PHIL)
* [Historical Player Graphs](http://yoori.space/hloroscopes/)
